In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

**Unity Catalog does not support creating tables from dbfs:/ or local paths.**

In [0]:
%sql
SELECT current_metastore() ,current_catalog(),    current_schema();

In [0]:
import sys
import os
# print(os.getcwd())
from  data_utils import *
# sys.path.append("/Workspace/Users/testing13cabs@gmail.com/spark/data_utils")

load_and_register_csv_as_table("dbfs:/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_2019-01.csv.gz","workspace","default","yellow_tripdata_source")




In [0]:
%sql
create or replace table yellow_tripdata_transformed as select *, to_date(tpep_pickup_datetime) as pickup_datetime, to_date(tpep_dropoff_datetime) as dropoff_datetime from yellow_tripdata_source limit 100

In [0]:
%sql
select * from yellow_tripdata_transformed limit 100

In [0]:
load_and_register_csv_as_table("dbfs:/databricks-datasets/nyctaxi/taxizone/taxi_zone_lookup.csv","workspace","default","zone_source")

In [0]:
%sql
create or replace table zone_transformed as select * except(LocationID) from zone_source limit 100;
    
select * from zone_transformed limit 100

In [0]:
%sql
create or replace table yellow_tripdata_sample as select * except(zone_source.LOcationID) from yellow_tripdata_transformed  left join  zone_source on zone_source.LocationID=yellow_tripdata_transformed.PULocationID 

In [0]:
%sql
describe formatted yellow_tripdata_sample

In [0]:
%sql
-- # Did not work
CREATE OR REPLACE TABLE yellow_trips_sample LOCATION "/tmp/datasets/datakickstart/yellow_trips_sample"
AS
SELECT * EXCEPT(taxi_zone.LocationID)
FROM yellow_tripdata_transformed trip
  LEFT JOIN zone_source taxi_zone
    ON trip.PULocationID = taxi_zone.LocationID

Delta and Iceberg is the only supported format for managed tables under Unity Catalog.

Option 1: Use USING DELTA (Recommended for managed tables)

Option 2: Use USING PARQUET with LOCATION (External table)

In [0]:
%sql
Drop table if exists yellow_trips_sample;
CREATE TABLE yellow_trips_sample using delta
AS
SELECT * EXCEPT(taxi_zone.LocationID)
FROM yellow_tripdata_transformed trip
  LEFT JOIN zone_source taxi_zone
    ON trip.PULocationID = taxi_zone.LocationID

In [0]:
%sql
Drop table if exists yellow_trips_sample;
CREATE TABLE yellow_trips_sample using iceberg
AS
SELECT * EXCEPT(taxi_zone.LocationID)
FROM yellow_tripdata_transformed trip
  LEFT JOIN zone_source taxi_zone
    ON trip.PULocationID = taxi_zone.LocationID